In [1]:
import pandas as pd
import numpy as np
#pd.options.display

In [3]:
CreditData = pd.read_excel('E:/Desktop/Github/Credit Risk Management on Loan Issuance/CreditData.xls', index_col = 0)
print(CreditData.head())
print(CreditData.describe())

    Status  LengthOfEmployment   Home  Age     Sex MaritalStatus Records  \
No                                                                         
1        0                   5  other   23  female        single      no   
2        0                  23   rent   47    male        single      no   
3        0                  19  other   55    male       married      no   
4        0                  12   rent   37  female       married     yes   
5        0                   3  other   21  female        single     yes   

      Job  Expenses  Income  Assets  Debt  LoanAmount  DurationOfLoan  
No                                                                     
1   fixed        35     120       0     0        1300             5.0  
2   fixed        44      86       0     0         350             2.0  
3   fixed        45     118    4000     0        1300             5.0  
4   fixed        86     110       0     0         700             5.0  
5   fixed        35      75       0

In [4]:
nData = len(CreditData.index)
nTrain = round(nData * 0.67)
CreditTrain = CreditData[:nTrain]
CreditTest = CreditData[nTrain:]

In [5]:
logit_y = 'Status~'
for i in CreditTrain.columns:
    if i != 'Status':
        logit_y += '+' + i
logit_y

'Status~+LengthOfEmployment+Home+Age+Sex+MaritalStatus+Records+Job+Expenses+Income+Assets+Debt+LoanAmount+DurationOfLoan'

In [6]:
import statsmodels.formula.api as smf
yTrain = CreditTrain['Status']

# Change to dummy variable
xTrain = CreditTrain.drop(['Status'], axis = 1)

In [7]:
model = smf.logit(logit_y, data = CreditTrain).fit()
print(model.summary())
print(model.params)
print(model.pvalues)

Optimization terminated successfully.
         Current function value: 0.453889
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 Status   No. Observations:                 2979
Model:                          Logit   Df Residuals:                     2960
Method:                           MLE   Df Model:                           18
Date:                Tue, 23 Feb 2021   Pseudo R-squ.:                  0.2373
Time:                        13:24:28   Log-Likelihood:                -1352.1
converged:                       True   LL-Null:                       -1772.9
                                        LLR p-value:                4.710e-167
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -1.5299      0.384     -3.982      0.000      -2.283

In [12]:
# Run and store wald_test on the whole datasetand find out non-significant parameters
wald_values = model.wald_test_terms().table
wald_values

,statistic,pvalue,df_constraint
Intercept,15.856057,6.834743006936223e-05,1
Home,39.047532,3.3184564535793915e-09,2
Sex,0.073828,0.7858425557456905,1
MaritalStatus,27.787394,4.025033691799671e-06,3
Records,194.410018,3.4655790501962964e-44,1
Job,95.590799,1.378300291652316e-20,3
LengthOfEmployment,81.125949,2.117807661504381e-19,1
Age,5.037043,0.02481081087300651,1
Expenses,29.437458,5.775057735666685e-08,1
Income,76.916163,1.7837130152681534e-18,1


In [13]:
logit_y2 = 'Status~+LengthOfEmployment+Home+MaritalStatus+Records+Job+Expenses+Income+Assets+Debt+LoanAmount'
model_2 = smf.logit(logit_y2, data = CreditTrain).fit()
print(model_2.summary())
print(model_2.params)
print(model_2.pvalues)

Optimization terminated successfully.
         Current function value: 0.455223
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 Status   No. Observations:                 2979
Model:                          Logit   Df Residuals:                     2963
Method:                           MLE   Df Model:                           15
Date:                Tue, 23 Feb 2021   Pseudo R-squ.:                  0.2351
Time:                        13:28:03   Log-Likelihood:                -1356.1
converged:                       True   LL-Null:                       -1772.9
                                        LLR p-value:                5.876e-168
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.8478      0.295     -2.874      0.004      -1.426

In [14]:
#Re-run the wald_test, but only on the cleaned data
wald_values = model_2.wald_test_terms().table
wald_values

,statistic,pvalue,df_constraint
Intercept,8.257521,0.00405835187705181,1
Home,37.300654,7.948148713768364e-09,2
MaritalStatus,27.579513,4.450269159390369e-06,3
Records,196.244987,1.37815429686104e-44,1
Job,97.711565,4.824969965765579e-21,3
LengthOfEmployment,76.794556,1.8969976436611555e-18,1
Expenses,31.027943,2.5434024650110822e-08,1
Income,78.321718,8.755396239937548e-19,1
Assets,14.290615,0.00015664388465902682,1
Debt,19.099652,1.2406749344168264e-05,1


In [15]:
preds = model_2.predict(CreditTest)
Default = pd.Series([ 1 if x > 0.5 else 0 for x in preds], index = CreditTest.index)

In [16]:
CreditTest['Prediction'] = preds
CreditTest['Status_pred'] = Default
CreditTest

C:\Users\hp\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\hp\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Status,LengthOfEmployment,Home,Age,Sex,MaritalStatus,Records,Job,Expenses,Income,Assets,Debt,LoanAmount,DurationOfLoan,Prediction,Status_pred
No,,,,,,,,,,,,,,,,
2980,0,5,owner,34,female,married,no,fixed,45,137,20000,0,3000,3.0,0.320128,0
2981,0,10,other,36,male,married,no,fixed,90,172,3000,0,1100,5.0,0.194387,0
2982,0,12,owner,28,male,married,no,fixed,45,148,3500,144,1350,5.0,0.073184,0
2983,0,35,owner,55,female,married,no,freelance,60,84,13000,0,1390,4.0,0.042300,0
2984,0,10,owner,40,male,married,no,freelance,90,301,2000,0,1100,5.0,0.094711,0
2985,0,11,rent,29,female,single,no,fixed,44,93,0,0,700,1.0,0.153646,0
2986,1,0,owner,64,male,widow,no,others,35,100,8000,0,1980,5.0,0.635568,1
2987,1,3,owner,26,female,single,no,freelance,35,169,60000,3000,800,3.0,0.041780,0
2988,1,20,owner,43,female,married,yes,fixed,45,120,10000,0,2000,5.0,0.315875,0


In [17]:
# Accuracy of the model
accuracy = round(CreditTest[CreditTest['Status_pred'] == CreditTest['Status']]['Status'].count() / CreditTest['Status'].count(), 6)
accuracy

0.79959100000000005

In [19]:
applicant = pd.read_excel('E:/Desktop/Github/Credit Risk Management on Loan Issuance/CreditApplicants.xls', index_col = 0)
print(applicant)

             LengthOfEmployment   Home  Age     Sex MaritalStatus Records  \
ApplicantNo                                                                 
1                             0  owner   28  female       married      no   
2                            21  other   51    male       married      no   
3                             0   rent   37  female       married      no   
4                            12  owner   64  female        single     yes   
5                             2  owner   33  female       married      no   
6                             3  owner   30    male      divorced      no   
7                             9   rent   40  female        single      no   
8                             4  other   50  female       married      no   

                   Job  Expenses  Income  Assets  Debt  LoanAmount  \
ApplicantNo                                                          
1                fixed        45      80    1500     0        1260   
2                fi

In [20]:
applicant_preds = model_2.predict(applicant)
applicant['Default_prob'] = applicant_preds
Approval = pd.Series([ 1 if x < 0.25 else 0 for x in applicant_preds], index = applicant.index)
applicant['status_pred'] = Approval
applicant

,LengthOfEmployment,Home,Age,Sex,MaritalStatus,Records,Job,Expenses,Income,Assets,Debt,LoanAmount,DurationOfLoan,Default_prob,status_pred
ApplicantNo,,,,,,,,,,,,,,,
1,0,owner,28,female,married,no,fixed,45,80,1500,0,1260,4.0,0.226205,1
2,21,other,51,male,married,no,fixed,45,140,3000,0,1200,5.0,0.063393,1
3,0,rent,37,female,married,no,others,88,130,0,0,250,1.0,0.501565,0
4,12,owner,64,female,single,yes,freelance,60,341,4500,0,900,3.0,0.170451,1
5,2,owner,33,female,married,no,freelance,75,220,16000,500,3000,3.0,0.570603,0
6,3,owner,30,male,divorced,no,fixed,60,157,15000,0,1000,5.0,0.238654,1
7,9,rent,40,female,single,no,fixed,44,91,0,0,1000,5.0,0.225956,1
8,4,other,50,female,married,no,freelance,60,150,7000,0,900,3.5,0.265615,0


In [21]:
#!pip3 uninstall statsmodels -y
#!pip3 install statsmodels==0.10.0rc2 --pre --user